# The Continuity Equation: One-Dimensional Advection of a Density Profile


**Physical Scenario**

Imagine a pollutant being introduced into a river flowing at a constant speed $u$. The pollutant's concentration $\rho(x, t)$ varies along the length of the river $x$ and changes over time $t$ due to the flow of water.

**Mathematical Model**

The one-dimensional continuity equation for this scenario is:

$$
\frac{\partial \rho}{\partial t} + \frac{\partial J}{\partial x} = 0
$$

- $\rho(x, t)$: Density (concentration) of the pollutant.
- $J(x, t)$: Flux of the pollutant.

Assuming the flux $J$ is proportional to the density and the constant velocity $u$:

$$
J = \rho u
$$

Substituting $J$ into the continuity equation:

$$
\frac{\partial \rho}{\partial t} + \frac{\partial (\rho u)}{\partial x} = 0
$$

We consider the case when $u$ is constant:

$$
\frac{\partial \rho}{\partial t} + u \frac{\partial \rho}{\partial x} = 0
$$

This is the **advection equation**, which describes how the density profile propagates over time due to the flow.

https://en.wikipedia.org/wiki/Advection

---

In [23]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [24]:
# Parameters
L = 10.0          # Length of the river (meters)
T = 5.0           # Total simulation time (seconds)
u = 1.0           # Flow velocity (m/s)

In [25]:
# Spatial and temporal discretization
nx = 200          # Number of spatial points
nt = 200          # Number of time steps
dx = L / (nx - 1)
dt = T / (nt - 1)
x = np.linspace(0, L, nx)
t = np.linspace(0, T, nt)

# CFL condition for stability
CFL = u * dt / dx
if CFL > 1:
    print("Warning: CFL condition not met. The simulation may be unstable.")

# Initial density profile (Gaussian distribution)
def initial_density(x):
    return np.exp(-((x - L/4)**2) / 0.1)

rho = initial_density(x)

In mathematics, the convergence condition by Courant–Friedrichs–Lewy (CFL) is a necessary condition for convergence while solving certain partial differential equations (usually hyperbolic PDEs) numerically. It arises in the numerical analysis of explicit time integration schemes, when these are used for the numerical solution. As a consequence, the time step must be less than a certain upper bound, given a fixed spatial increment, in many explicit time-marching computer simulations; otherwise, the simulation produces incorrect or unstable results. The condition is named after Richard Courant, Kurt Friedrichs, and Hans Lewy who described it in their 1928 paper.[1]

https://en.wikipedia.org/wiki/Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition

In [26]:
# Initial Pollutant Concentration
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=rho,
    mode='lines',
    name='Initial Density'
))

fig.update_layout(
    title='Initial Pollutant Concentration',
    xaxis_title='Position along the River (m)',
    yaxis_title='Pollutant Concentration',
    autosize=True,
    height=500
)

fig.show()

In [27]:
# Store density profiles at selected time steps for visualization
rho_history = [rho.copy()]
time_steps = [0]

# Finite Difference Scheme (Upwind method)
for n in range(1, nt):
    rho_new = rho.copy()
    for i in range(1, nx):
        rho_new[i] = rho[i] - CFL * (rho[i] - rho[i - 1])
    rho = rho_new

    # Store data every 20 time steps
    if n % 20 == 0:
        rho_history.append(rho.copy())
        time_steps.append(t[n])

**Upwind method** overview:

https://en.wikipedia.org/wiki/Upwind_scheme

In [28]:
# Visualization using Plotly
fig = go.Figure()

for idx, rho_snapshot in enumerate(rho_history):
    fig.add_trace(go.Scatter(
        x=x,
        y=rho_snapshot,
        mode='lines',
        name=f't = {time_steps[idx]:.2f} s'
    ))

fig.update_layout(
    title='Advection of Pollutant Concentration Over Time',
    xaxis_title='Position along the River (m)',
    yaxis_title='Pollutant Concentration',
    legend_title='Time Steps',
    autosize=True,
    height=500
)

fig.show()


**IMPORTANT: This result is not correct! **

**It is affected by the numerical diffusion. In general, the shape of the figure should not be changed in this specific example.**

The discrepancy arises from the numerical method used to solve the advection equation. Specifically, the Upwind Finite Difference Method is a first-order scheme, which inherently introduces numerical diffusion.

**Numerical Diffusion Explained**
- First-Order Accuracy: The Upwind scheme approximates derivatives using first-order differences, which are less accurate compared to higher-order methods.
- Artificial Spreading: This lower accuracy causes the numerical solution to diffuse the concentration profile artificially, leading to a loss of sharpness in gradients. As a result:
    - Peak Concentration: Decreases because the high-density region spreads out.
    - Width: Increases as the pollutant disperses over a larger area.
- https://en.wikipedia.org/wiki/Numerical_diffusion

We used the following parameters:

nx = 200 (Number of spatial points)

nt = 200 (Number of time steps)

Let's increast nx and nt, recalculate the solution, and compare new graphical representation with a new one.

In [29]:
# Spatial and temporal discretization
nx = 1000          # Number of spatial points
nt = 1000          # Number of time steps
dx = L / (nx - 1)
dt = T / (nt - 1)
x = np.linspace(0, L, nx)
t = np.linspace(0, T, nt)

# CFL condition for stability
CFL = u * dt / dx
if CFL > 1:
    print("Warning: CFL condition not met. The simulation may be unstable.")

rho = initial_density(x)

In [30]:
# Store density profiles at selected time steps for visualization
rho_history = [rho.copy()]
time_steps = [0]

# Finite Difference Scheme (Upwind method)
for n in range(1, nt):
    rho_new = rho.copy()
    for i in range(1, nx):
        rho_new[i] = rho[i] - CFL * (rho[i] - rho[i - 1])
    rho = rho_new

    # Store data every 100 time steps
    if n % 100 == 0:
        rho_history.append(rho.copy())
        time_steps.append(t[n])

In [31]:

# Visualization using Plotly
fig = go.Figure()

for idx, rho_snapshot in enumerate(rho_history):
    fig.add_trace(go.Scatter(
        x=x,
        y=rho_snapshot,
        mode='lines',
        name=f't = {time_steps[idx]:.2f} s'
    ))

fig.update_layout(
    title='Advection of Pollutant Concentration Over Time',
    xaxis_title='Position along the River (m)',
    yaxis_title='Pollutant Concentration',
    legend_title='Time Steps',
    autosize=True,
    height=500
)

fig.show()


**More accurate result**

nx = 1000 (Number of spatial points)

nt = 1000 (Number of time steps)

However, even for this result we can see that the shape of the original Gaussian is changed over time. 

**Let's review the general theory and understand why the shape should not be changed, and then we will revisit our numerical approach to decrease the numerical diffusion.**

---

### The shape of the function remains unchanged as it propagates

Let's delve into how substituting a function of the form $ \rho(x - ut) $ into the **advection equation** demonstrates that the shape of the function remains unchanged as it propagates. This is a fundamental property of solutions to the advection equation, often interpreted as **traveling wave solutions**.

**1. The Advection Equation Recap**

The **linear advection equation** with constant speed $ u $ in one spatial dimension is given by:

$$
\frac{\partial \rho}{\partial t} + u \frac{\partial \rho}{\partial x} = 0
$$

where:
- $ \rho = \rho(x, t) $ is the transported quantity (e.g., concentration, temperature),
- $ u $ is the constant advection (transport) speed,
- $ x $ is the spatial coordinate,
- $ t $ is time.

**2. Proposing a Traveling Wave Solution**

We propose a solution of the form:

$$
\rho(x, t) = f(\xi) \quad \text{where} \quad \xi = x - ut
$$

Here, $ f $ is an arbitrary differentiable function representing the shape of the wave, and $ \xi $ is a **moving coordinate** that shifts with speed $ u $.

**Interpretation:**
- **$ f(\xi) $:** Represents the wave's shape.
- **$ \xi = x - ut $:** As time $ t $ increases, $ \xi $ adjusts so that the wave maintains its shape while moving in the positive $ x $-direction at speed $ u $.

**3. Substituting $ \rho(x, t) = f(x - ut) $ into the Advection Equation**

Let's verify that this proposed solution satisfies the advection equation.

**Step 1: Compute Partial Derivatives**

1. **Partial derivative with respect to time $ t $:**

$$
\frac{\partial \rho}{\partial t} = \frac{d f}{d \xi} \cdot \frac{\partial \xi}{\partial t} = f'(\xi) \cdot (-u) = -u f'(\xi)
$$

2. **Partial derivative with respect to space $ x $:**

$$
\frac{\partial \rho}{\partial x} = \frac{d f}{d \xi} \cdot \frac{\partial \xi}{\partial x} = f'(\xi) \cdot 1 = f'(\xi)
$$

*Here, $ f'(\xi) $ denotes the derivative of $ f $ with respect to its argument $ \xi $.*

**Step 2: Substitute into the Advection Equation**

$$
\frac{\partial \rho}{\partial t} + u \frac{\partial \rho}{\partial x} = -u f'(\xi) + u f'(\xi) = 0
$$

**Conclusion:**

The left-hand side of the advection equation simplifies to zero, which satisfies the equation identically. Therefore, $ \rho(x, t) = f(x - ut) $ is indeed a solution to the advection equation.

**4. Implications of the Solution**

**a. Shape Preservation**

Since $ f(\xi) $ remains unchanged as $ \xi = x - ut $ varies, the **shape of the wave** $ f $ is preserved over time. The entire profile $ f $ simply translates along the $ x $-axis without distortion.

**b. Wave Propagation**

The solution represents a **traveling wave** moving in the positive $ x $-direction with speed $ u $. If $ u $ is negative, the wave travels in the negative $ x $-direction, and the solution can be written as $ \rho(x, t) = f(x + |u|t) $.

**c. Superposition Principle**

Given the linearity of the advection equation, any linear combination of traveling wave solutions is also a solution. For example:

$$
\rho(x, t) = f(x - ut) + g(x + ut)
$$

where $ f $ and $ g $ are arbitrary differentiable functions, represents waves traveling in opposite directions.

**5. Visual Illustration**

Imagine a **pulse** or **wave packet** described by $ f(\xi) $:

- **Initial Condition at $ t = 0 $:** $ \rho(x, 0) = f(x) $
- **At $ t = t_0 $:** $ \rho(x, t_0) = f(x - ut_0) $

This shows that the entire profile of $ f $ has shifted to the right by $ u t_0 $ units without any change in its shape or amplitude.

**6. Generalizations and Extensions**

While $ \rho(x - ut) $ is a simple traveling wave solution, more complex solutions can involve superpositions, discontinuities (shock waves), or even multi-dimensional traveling waves, depending on the context and the nature of the function $ f $.

**Nonlinear Advection:**
In nonlinear advection equations, where the speed $ u $ may depend on $ \rho $ itself (e.g., $ \frac{\partial \rho}{\partial t} + u(\rho) \frac{\partial \rho}{\partial x} = 0 $), the solutions can exhibit more complex behaviors like wave steepening and shock formation.

**7. Conclusion**

By substituting a function of the form $ \rho(x - ut) $ into the linear advection equation with constant speed $ u $, we demonstrate that:

1. **The proposed function satisfies the equation identically.**
2. **The shape of the function $ f $ is preserved over time.**
3. **The solution represents a traveling wave moving at speed $ u $ without distortion.**

This fundamental property makes the advection equation a powerful model for understanding transport phenomena where the shape of the moving quantity remains unchanged as it propagates through space.


---

### Understanding the Advection Example

In the current example, we model how the concentration (or density) of a pollutant in a river evolves over time. The pollutant is transported by the river's flow at a constant velocity. This process is described by the **advection equation**, a simplified form of the continuity equation.

---

**Physical Scenario**

- **Pollutant in a River**: Imagine a pollutant introduced into a river flowing at a constant velocity $u$.
- **One-Dimensional Flow**: We model the river as a one-dimensional domain along the $x$-axis.
- **Density Function**: The pollutant concentration is represented by $\rho(x, t)$, which depends on position $x$ and time $t$.

---

**Key Assumptions**

- **No Sources or Sinks**: There's no addition or removal of pollutant within the river.
- **No Diffusion or Dispersion**: The pollutant does not spread out due to molecular diffusion or turbulent mixing.
- **Constant Flow Velocity**: The river flows at a steady speed $u$, and this velocity does not change over time or position.

---

**Connection to the Differential Equation**

**The Advection Equation**

The continuity equation simplifies to the **advection equation** under the given assumptions:

$$
\frac{\partial \rho}{\partial t} + u \frac{\partial \rho}{\partial x} = 0
$$

- $\frac{\partial \rho}{\partial t}$: The rate of change of pollutant concentration at a fixed point in space.
- $u \frac{\partial \rho}{\partial x}$: The rate at which pollutant concentration is transported by the flow.

---

**Physical Interpretation**

- The advection equation states that the change in concentration at a point over time is due to the movement of concentration gradients by the flow.
- If there is a higher concentration upstream ($x - \Delta x$) compared to downstream ($x$), the pollutant will be carried to $x$, increasing the concentration there.

---

**Analytical Solution**

**Method of Characteristics**

The advection equation is a first-order linear partial differential equation that can be solved using the method of characteristics.

**General Solution**:

$$
\rho(x, t) = f(x - u t)
$$

- $f$: The initial concentration profile at time $t = 0$.
- **Interpretation**: The initial profile moves to the right (downstream) without changing shape at a constant speed $u$.

**Verification**

To verify that $\rho(x, t) = f(x - u t)$ is a solution:

1. **Compute Time Derivative**:

   $$
   \frac{\partial \rho}{\partial t} = \frac{d f}{d (x - u t)} \cdot (-u) = -u f'(x - u t)
   $$

2. **Compute Spatial Derivative**:

   $$
   \frac{\partial \rho}{\partial x} = \frac{d f}{d (x - u t)} \cdot 1 = f'(x - u t)
   $$

3. **Substitute into the Advection Equation**:

   $$
   \frac{\partial \rho}{\partial t} + u \frac{\partial \rho}{\partial x} = (-u f') + u (f') = 0
   $$

- **Conclusion**: The expression satisfies the advection equation.

---

**Understanding the Graph**

**Movement of the Density Profile**

- **Initial Profile**: At $t = 0$, the concentration is a Gaussian centered at $x = x_0$.
- **Evolution Over Time**: As time progresses, the entire Gaussian profile shifts to the right without changing shape.
- **Constant Speed**: The profile moves at speed $u$, consistent with the solution $\rho(x, t) = f(x - u t)$.

---

**Connection to the Differential Equation**

- **Advection-Driven Transport**: The advection equation predicts that the concentration at a point $x$ at time $t$ is the same as the initial concentration at $x - u t$.
- **Shape Preservation**: Since there are no terms in the equation accounting for diffusion, the shape remains unchanged.
- **Mass Conservation**: The area under the concentration curve remains constant, indicating that the total amount of pollutant is conserved.

---

**Numerical Simulation**

**Finite Difference Method**

- **Discretization**: Space and time are discretized into small intervals ($\Delta x$ and $\Delta t$).
- **Upwind Scheme**: An explicit method suitable for solving hyperbolic PDEs like the advection equation.

**Update Formula**:

$$
\rho_i^{n+1} = \rho_i^n - \frac{u \Delta t}{\Delta x} (\rho_i^n - \rho_{i-1}^n)
$$

- $\rho_i^n$: Concentration at position $x_i$ and time $t_n$.
- **Courant Number (CFL)**:

  $$
  \text{CFL} = \frac{u \Delta t}{\Delta x}
  $$

  - Must satisfy $\text{CFL} \leq 1$ for numerical stability.

- https://en.wikipedia.org/wiki/Upwind_scheme
---

**How It Solves the Advection Equation**

- **Approximation of Derivatives**:

  - **Temporal Derivative**:

    $$
    \frac{\partial \rho}{\partial t} \approx \frac{\rho_i^{n+1} - \rho_i^n}{\Delta t}
    $$

  - **Spatial Derivative**:

    $$
    \frac{\partial \rho}{\partial x} \approx \frac{\rho_i^n - \rho_{i-1}^n}{\Delta x}
    $$

- **Substituting into the Advection Equation**:

  $$
  \frac{\rho_i^{n+1} - \rho_i^n}{\Delta t} + u \frac{\rho_i^n - \rho_{i-1}^n}{\Delta x} = 0
  $$

- **Rearranging**:

  $$
  \rho_i^{n+1} = \rho_i^n - \text{CFL} (\rho_i^n - \rho_{i-1}^n)
  $$

- **Interpretation**:

  - The new concentration at $x_i$ depends on its current value and the difference between its value and that at the upstream point $x_{i-1}$.
  - This captures the upstream influence, consistent with the direction of advection.

---

**Visualization**

- **Profiles Over Time**: The simulation records the concentration profile at various time steps.
- **Graph Movement**: The plotted profiles show the initial Gaussian shifting to the right over time.
- **Consistency with Theory**: The numerical results align with the analytical solution of the advection equation.

---

**Connecting Differential Equation and Graph**

**Role of the Differential Equation**

- **Predicts Behavior**: The advection equation dictates how the concentration changes in both space and time.
- **Shape and Movement**: It predicts that the shape of the concentration profile remains constant while moving at speed $u$.

---

**Numerical Simulation as an Approximation**

- **Discretization Errors**: While the finite difference method approximates derivatives, small errors can accumulate.
- **CFL Condition**: Ensures stability and accuracy of the simulation.
- **Visualization of Solutions**: The simulation provides a visual representation of the mathematical solution.

---


### Solutions to Mitigate Numerical Diffusion

To better preserve the Gaussian shape in the simulation, consider the following approaches:

1. **Higher-Order Numerical Schemes**
   - **Lax-Wendroff Method**: A second-order accurate scheme that reduces numerical diffusion.
   - **MacCormack Method**: Another second-order method suitable for hyperbolic PDEs like the advection equation.
   - **Essentially Non-Oscillatory (ENO) Schemes**: High-order schemes that handle sharp gradients without introducing significant diffusion.
   
2. **Refining Discretization Parameters**
   - Increase Spatial Resolution ($nx$): Finer spatial grids can capture gradients more accurately.
   - Decrease Time Step ($nt$): Smaller time steps reduce the Courant–Friedrichs–Lewy (CFL) number, potentially enhancing stability and accuracy.

3. **Implementing Anti-Diffusion Techniques**
   - **Flux Limiters**: Modify the numerical flux to prevent excessive diffusion.
   - **Total Variation Diminishing (TVD) Schemes**: Ensure that the total variation of the solution does not increase, preserving sharp fronts.


### Implementing a Higher-Order Scheme: Lax-Wendroff Method

To address the numerical diffusion observed with the Upwind scheme, let's implement the Lax-Wendroff Method, which is second-order accurate in both space and time. This method provides a better approximation of the advection equation and preserves the shape of the Gaussian profile more effectively.

**Lax-Wendroff Scheme Overview**

The Lax-Wendroff scheme updates the density $\rho$ using both first and second spatial derivatives, effectively capturing the wave propagation with reduced numerical diffusion.

**Update Formula**:

$$
\rho_i^{n+1} = \rho_i^n - \frac{u \Delta t}{2 \Delta x} (\rho_{i+1}^n - \rho_{i-1}^n) + \frac{u^2 \Delta t^2}{2 \Delta x^2} (\rho_{i+1}^n - 2 \rho_i^n + \rho_{i-1}^n)
$$




In [32]:
# Parameters
L = 10.0          # Length of the river (meters)
T = 5.0           # Total simulation time (seconds)
nx = 1000          # Number of spatial points
nt = 1000          # Number of time steps
u = 1.0           # Flow velocity (m/s)

# Spatial and temporal discretization
dx = L / (nx - 1)
dt = T / (nt - 1)
x = np.linspace(0, L, nx)
t = np.linspace(0, T, nt)

# CFL condition for stability
CFL = u * dt / dx
if CFL > 1:
    print("Warning: CFL condition not met. The simulation may be unstable.")

rho = initial_density(x)

# Store density profiles at selected time steps for visualization
rho_history = [rho.copy()]

time_steps = [0]

# Lax-Wendroff Scheme
for n in range(1, nt):
    rho_new = rho.copy()
    for i in range(1, nx-1):
        rho_new[i] = (rho[i] - 
                      0.5 * CFL * (rho[i+1] - rho[i-1]) + 
                      0.5 * CFL**2 * (rho[i+1] - 2*rho[i] + rho[i-1]))
    # Boundary conditions (Periodic)
    rho_new[0] = rho_new[-2]
    rho_new[-1] = rho_new[0]
    
    rho = rho_new.copy()
    
    # Store data every 100 time steps
    if n % 100 == 0:
        rho_history.append(rho.copy())
        time_steps.append(t[n])

In [33]:
# Visualization using Plotly
fig = go.Figure()

for idx, rho_snapshot in enumerate(rho_history):
    fig.add_trace(go.Scatter(
        x=x,
        y=rho_snapshot,
        mode='lines',
        name=f't = {time_steps[idx]:.2f} s'
    ))

fig.update_layout(
    title='Advection of Pollutant Concentration Over Time (Lax-Wendroff)',
    xaxis_title='Position along the River (m)',
    yaxis_title='Pollutant Concentration',
    legend_title='Time Steps',
    autosize=True,
    height=500
)

fig.show()

**Explanation of the Updated Code**

1.  **Lax-Wendroff Update**: The core update loop implements the Lax-Wendroff scheme, incorporating both first and second spatial derivatives to improve accuracy.
    
2.  **Boundary Conditions**: Implemented **periodic boundary conditions** to allow the Gaussian to exit one end and enter the other, minimizing edge effects. You can adjust this based on your physical scenario.
    
3.  **Visualization**: The Plotly graph now reflects the reduced numerical diffusion, showing the Gaussian maintaining its shape as it advects downstream.

---

**Expected Results**

With the Lax-Wendroff scheme:

- **Shape Preservation**: The Gaussian maintains its peak and width more accurately over time.
- **Reduced Numerical Diffusion**: Less artificial spreading and peak attenuation compared to the Upwind scheme.
- **Mass Conservation**: The total area under the Gaussian remains approximately constant, indicating better mass conservation.


---

### Summary of the Shape Preservation in the Advection Equation

**1. Analyzing the Advection Equation Itself**

The **advection equation** is a simplified form of the continuity equation, focusing solely on the transport of a conserved quantity without any sources, sinks, or diffusion. The equation is given by:

$$
\frac{\partial \rho}{\partial t} + u \frac{\partial \rho}{\partial x} = 0
$$

Where:
- $\rho(x, t)$ is the **density** or **concentration** of the conserved quantity at position $x$ and time $t$.
- $u$ is the **constant flow velocity**.

**Key Characteristics of the Advection Equation:**

1. **Linearity**:
   - The equation is **linear** in $\rho$, meaning that if $\rho_1$ and $\rho_2$ are solutions, then any linear combination $a \rho_1 + b \rho_2$ is also a solution.
   
2. **First-Order Partial Differential Equation (PDE)**:
   - It involves first-order derivatives in both time and space.
   
3. **Hyperbolic Nature**:
   - Being a hyperbolic PDE, it models wave propagation and transport phenomena where information travels along characteristic lines.

4. **No Diffusion or Dispersion Terms**:
   - Unlike equations that include second-order spatial derivatives (e.g., diffusion equation), the advection equation lacks terms that cause spreading or smoothing of the solution.

**Implications of These Characteristics:**

- **Shape Preservation**:
  - Since there are no diffusion or dispersion terms to cause spreading, the **shape** of the initial density profile is preserved as it moves.
  
- **Translation of the Initial Profile**:
  - The entire profile translates downstream at a constant velocity $u$ without any deformation.

---

**2. Role of Initial Conditions**

While the **advection equation** dictates **how** the density profile moves, the **initial conditions** define **what** is being moved. The solution to the advection equation depends on both the equation itself and the initial profile provided.

**General Solution Using Method of Characteristics:**

The method of characteristics transforms the PDE into ordinary differential equations (ODEs) along characteristic lines where the solution is constant.

**Characteristic Lines**:

$$
\frac{dx}{dt} = u
$$

**Solution Along Characteristics**:

$$
\rho(x, t) = f(x - u t)
$$

Where $f$ is the **initial concentration profile** at $t = 0$:

$$
\rho(x, 0) = f(x)
$$

**Interpretation**:

- **Translation Without Change**:
  - The function $f(x - u t)$ represents the initial profile $f(x)$ shifted by $u t$ units to the right (assuming $u > 0$).
  
- **No Change in Shape**:
  - Since $f$ is merely shifted and not altered in form, the **shape** of the density profile remains unchanged over time.

---

**3. Visual Confirmation**

Consider an initial Gaussian density profile:

$$
\rho(x, 0) = e^{-\frac{(x - x_0)^2}{2\sigma^2}}
$$

**Solution at Time $t$:**

$$
\rho(x, t) = e^{-\frac{(x - x_0 - u t)^2}{2\sigma^2}}
$$

**Observations:**

- **Mean Shift**:
  - The peak of the Gaussian moves to $x = x_0 + u t$.
  
- **Variance Remains Constant**:
  - The spread ($\sigma$) of the Gaussian remains unchanged, indicating no diffusion.

- **Shape Unaltered**:
  - The Gaussian retains its bell shape as it translates downstream.

---